In [1]:
import os
os.chdir('../')
os.getcwd()

'/Users/dennis/repos/study/mt/mt_rdf'

In [2]:
import pandas as pd
from tqdm import tqdm
from functools import reduce

In [3]:
virt_orig_df = pd.read_csv('results/wdbench/ppaths/virt/results_ppaths_original.csv')
virt_orig_df_forced = pd.read_csv('results/wdbench/ppaths/virt/results_ppaths_original_forced.csv')
virt_orig_df_opt = pd.read_csv('results/wdbench/ppaths/virt/results_ppaths_opt_virt.csv')
virt_orig_df.drop('Unnamed: 0', axis=1, inplace=True)
virt_orig_df_forced.drop('Unnamed: 0', axis=1, inplace=True)
virt_orig_df_opt.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
virt_merged_df = pd.merge(virt_orig_df, virt_orig_df_forced, how='left', left_on=['query_id', 'exec_n'], right_on=['query_id', 'exec_n'], suffixes=('', '_forced'))
virt_merged_df = pd.merge(virt_merged_df, virt_orig_df_opt, how='left', left_on=['query_id', 'exec_n'], right_on=['query_id', 'exec_n'], suffixes=('', '_opt'))

In [5]:
virt_merged_df['exec_time'].sum() / 4

1719.3866250514984

In [6]:
virt_merged_df

,query_id,exec_n,exec_time,results,exec_time_forced,results_forced,exec_time_opt,results_opt
0,P365,0,0.034540,NaN,50.981081,NaN,0.037571,NaN
1,P365,1,0.005320,NaN,50.173872,NaN,0.008669,NaN
2,P365,2,0.004696,NaN,50.759371,NaN,0.006482,NaN
3,P365,3,0.003977,NaN,50.263360,NaN,0.004403,NaN
4,P376,0,0.030846,NaN,49.986881,NaN,0.006958,NaN
...,...,...,...,...,...,...,...,...
851,P279,3,0.270709,NaN,432.549118,NaN,NaN,NaN
852,P263,0,0.540939,NaN,5.716760,NaN,NaN,NaN
853,P263,1,0.514997,NaN,5.554682,NaN,NaN,NaN
854,P263,2,0.518899,NaN,5.505023,NaN,NaN,NaN


In [7]:
virt_merged_df[(virt_merged_df['exec_time'] >= virt_merged_df['exec_time_forced']) & virt_merged_df['exec_time_forced'] != 0]

,query_id,exec_n,exec_time,results,exec_time_forced,results_forced,exec_time_opt,results_opt
8,P230,0,295.505047,NaN,42.569398,NaN,NaN,NaN
9,P230,1,246.681965,NaN,43.057276,NaN,NaN,NaN
10,P230,2,246.280662,NaN,43.482172,NaN,NaN,NaN
11,P230,3,247.935213,NaN,44.602695,NaN,NaN,NaN
40,P355,0,3.443652,NaN,1.117381,NaN,0.004738,NaN
...,...,...,...,...,...,...,...,...
759,P275,3,60.404140,NaN,6.054290,NaN,NaN,NaN
784,P285,0,4.393668,NaN,0.192304,NaN,0.021630,NaN
785,P285,1,4.353490,NaN,0.167747,NaN,0.016577,NaN
786,P285,2,4.369842,NaN,0.170515,NaN,0.015898,NaN


In [8]:
virt_opt_not_opt = virt_merged_df[['query_id',  'exec_time', 'exec_time_forced']]

In [9]:
virt_opt_not_opt.loc[virt_opt_not_opt['exec_time_forced'] == 0, 'exec_time_forced'] = 900

In [10]:
virt_opt_not_opt[virt_opt_not_opt['exec_time_forced'] == 900]

,query_id,exec_time,exec_time_forced
32,P357,34.484950,900.0
33,P357,30.141387,900.0
34,P357,30.688312,900.0
35,P357,30.222399,900.0
44,P385,1.028157,900.0
...,...,...,...
839,P315,0.070925,900.0
844,P283,2.621537,900.0
845,P283,2.607222,900.0
846,P283,2.565370,900.0


In [11]:
virt_opt_not_opt.groupby('query_id').apply(lambda x: x['exec_time'].mean() >= x['exec_time_forced'].mean())

query_id
P180    False
P181     True
P182    False
P183    False
P184    False
        ...  
P399    False
P400    False
P401    False
P402    False
P403    False
Length: 214, dtype: bool

In [12]:
dataset = virt_opt_not_opt.groupby('query_id').apply(lambda x: x['exec_time'].mean() >= x['exec_time_forced'].mean())

In [13]:
ds = dataset.to_frame().reset_index().rename(columns={0: 'label'})

In [14]:
ds['label'] = ds['label'].astype(int)

In [15]:
ds

,query_id,label
0,P180,0
1,P181,1
2,P182,0
3,P183,0
4,P184,0
...,...,...
209,P399,0
210,P400,0
211,P401,0
212,P402,0


In [13]:
def add_query_to_df(df, nl=False):
    ds_dict = {'query_id': [], 'query': [], 'label': []}
    for index, row in tqdm(df.iterrows()):
        filename = f'{row["query_id"]}.sparql'

        with open(os.path.join(f'data/raw/wdbench/ppaths/{"nl" if nl else "original"}', filename), 'r') as file:
            query = file.read()
            if nl:
                query = query.split('NL-QUESTION:')[1].strip()
                # print(repr(query))
            ds_dict['query'].append(query)
            ds_dict['query_id'].append(row['query_id'])
            ds_dict['label'].append(row['label'])
        
    final_ds = pd.DataFrame(ds_dict)
    return final_ds

In [20]:
nl_ds = add_query_to_df(ds, nl=True)

214it [00:00, 9866.89it/s]


In [21]:
nl_ds

,query_id,query,label
0,P180,Which municipal arrondissements are located in...,0
1,P181,What is the field of occupation of Stéphane Du...,1
2,P182,Which administrative territorial entities are ...,0
3,P183,What is the ISO 3166-2 code of the administrat...,0
4,P184,What is the ISO 3166-2 code of the administrat...,0
...,...,...,...
209,P399,Which paintings created by a member of the Roy...,0
210,P400,What are the locations and types of mountains ...,0
211,P401,What are the different types of musical instru...,0
212,P402,In which administrative territorial entities d...,0


In [22]:
final_ds = add_query_to_df(ds, nl=False)

214it [00:00, 10473.77it/s]


In [23]:
final_ds['query_len'] = final_ds.apply(lambda x: len(x['query']), axis=1)
nl_ds['query_len'] = nl_ds.apply(lambda x: len(x['query']), axis=1)

In [26]:
nl_ds['query_len'].max()

214

In [48]:
final_ds['query_len'].mean()

247.07009345794393

In [49]:
final_ds['query_len'].max()

684

In [55]:
nl_ds[nl_ds['query_len'] >= 512]

,query_id,query,label,query_len


In [32]:
final_ds[final_ds['query_len'] > 512].loc[80, 'query']

'SELECT * WHERE {\n\t?x1 <http://www.wikidata.org/prop/direct/P105> ?x2 .\n\t?x3 <http://www.wikidata.org/prop/direct/P105> ?x4 .\n\t?x1 <http://www.wikidata.org/prop/direct/P171> ?x3 .\n\t?x1 <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q16521> .\n\t?x3 <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q16521> .\n\t?x2 <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q427626> .\n\t?x4 <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q427626> .\n\t?x4 <http://www.wikidata.org/prop/direct/P361> ?x2 .\n\t?x1 <http://www.wikidata.org/prop/direct/P171>+ <http://www.wikidata.org/entity/Q130942> .\n}'

In [33]:
final_ds

,query_id,query,label,query_len
0,P180,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0,196
1,P181,SELECT * WHERE {\n\t<http://www.wikidata.org/e...,1,223
2,P182,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0,308
3,P183,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0,166
4,P184,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0,164
...,...,...,...,...
209,P399,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0,456
210,P400,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0,219
211,P401,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0,163
212,P402,SELECT * WHERE {\n\t?x1 <http://www.wikidata.o...,0,164


In [29]:
final_ds[['query_id', 'query', 'label']].to_csv('data/raw/ppaths_join.csv', index=False)

In [34]:
nl_ds[['query_id', 'query', 'label']].to_csv('data/raw/ppaths_join_nl_sparql.csv', index=False)

In [4]:
join_nl = pd.read_csv('data/raw/ppaths_join_nl.csv')

In [5]:
import torch

In [6]:
encodings = torch.load('data/processed/ppaths_join/ppaths_join_nl_encoded.pt')

In [10]:
len(encodings), len(join_nl)

(214, 214)

In [11]:
join_nl['encoding'] = encodings

In [12]:
join_nl

,query_id,query,label,encoding
0,P180,Which municipal arrondissements are located in...,0,"[tensor(0.0038), tensor(-0.0119), tensor(-0.00..."
1,P181,What is the field of occupation of Stéphane Du...,1,"[tensor(-0.0194), tensor(-0.0055), tensor(-0.0..."
2,P182,Which administrative territorial entities are ...,0,"[tensor(-0.0324), tensor(0.0054), tensor(0.005..."
3,P183,What is the ISO 3166-2 code of the administrat...,0,"[tensor(0.0134), tensor(0.0178), tensor(0.0019..."
4,P184,What is the ISO 3166-2 code of the administrat...,0,"[tensor(-0.0121), tensor(0.0084), tensor(0.005..."
...,...,...,...,...
209,P399,Which paintings created by a member of the Roy...,0,"[tensor(-0.0147), tensor(0.0059), tensor(0.002..."
210,P400,What are the locations and types of mountains ...,0,"[tensor(-0.0180), tensor(0.0007), tensor(-0.00..."
211,P401,What are the different types of musical instru...,0,"[tensor(0.0032), tensor(0.0048), tensor(-0.012..."
212,P402,In which administrative territorial entities d...,0,"[tensor(-0.0276), tensor(0.0100), tensor(-0.01..."


In [13]:
type(join_nl.iloc[0]['encoding'])

torch.Tensor

In [14]:
join_nl.to_csv('data/processed/ppaths_join/ppaths_join_nl_encoded.csv', index=False)

In [15]:
len(encodings[0])

4096

In [16]:
asdf = pd.read_csv('data/processed/wdbench/wdbench_nl_encoded.csv')

In [18]:
asdf.head

<bound method NDFrame.head of       query_id                                              query  \
0            1  What information is available about the movie ...   
1            2  What are the family name, date of birth, place...   
2            3  What are the dates of birth, places of birth, ...   
3            4  Which painters have a father who is also a pai...   
4            5  What are the male ski jumpers from Poland and ...   
...        ...                                                ...   
2653       677  What are the coordinates of airports located i...   
2654       678  What is the image, spouse, and date of birth o...   
2655       679          Which humans have published in something?   
2656       680            What films are instances of Ari Folman?   
2657       681         What is the library's coordinate location?   

         query_type  label                                           encoding  
0              opts     -1  tensor([-0.0022,  0.0454,  0.0139

In [22]:
asdf.iloc[0]['encoding']

'tensor([-0.0022,  0.0454,  0.0139,  ...,  0.0002, -0.0323,  0.0227])'

In [23]:
wdbench_encodings = torch.load('data/processed/wdbench/wdbench_nl_encoding.pt')

In [24]:
wdbench_encodings[0]

tensor([-0.0022,  0.0454,  0.0139,  ...,  0.0002, -0.0323,  0.0227])

In [25]:
asdf['encoding'] = wdbench_encodings

In [26]:
asdf.iloc[0]['encoding']

tensor([-0.0022,  0.0454,  0.0139,  ...,  0.0002, -0.0323,  0.0227])

In [27]:
asdf.to_csv('data/processed/wdbench/wdbench_nl_encoded.csv', index=False)

In [28]:
asdf['encoding'].max()

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [29]:
asdf['query_len'] = asdf.apply(lambda x: len(x['query']), axis=1)

In [30]:
asdf['query_len'].max()

294

In [32]:
join_nl['query_len'] = join_nl.apply(lambda x: len(x['query']), axis=1)

In [33]:
join_nl['query_len'].max()

214

In [34]:
asdf

,query_id,query,query_type,label,encoding,query_len
0,1,What information is available about the movie ...,opts,-1,"[tensor(-0.0022), tensor(0.0454), tensor(0.013...",107
1,2,"What are the family name, date of birth, place...",opts,-1,"[tensor(-0.0191), tensor(0.0182), tensor(-0.01...",128
2,3,"What are the dates of birth, places of birth, ...",opts,-1,"[tensor(-0.0073), tensor(0.0138), tensor(-0.01...",158
3,4,Which painters have a father who is also a pai...,opts,-1,"[tensor(-0.0227), tensor(-0.0154), tensor(0.00...",107
4,5,What are the male ski jumpers from Poland and ...,opts,-1,"[tensor(-0.0159), tensor(0.0058), tensor(-0.03...",73
...,...,...,...,...,...,...
2653,677,What are the coordinates of airports located i...,multiple_bgps,-1,"[tensor(-0.0076), tensor(-0.0131), tensor(-0.0...",56
2654,678,"What is the image, spouse, and date of birth o...",multiple_bgps,-1,"[tensor(-0.0444), tensor(-0.0048), tensor(-0.0...",76
2655,679,Which humans have published in something?,multiple_bgps,-1,"[tensor(-0.0298), tensor(-0.0123), tensor(-0.0...",41
2656,680,What films are instances of Ari Folman?,multiple_bgps,-1,"[tensor(-0.0359), tensor(0.0091), tensor(-0.01...",39


In [4]:
qwerty = pd.read_csv('data/processed/wdbench/wdbench_nl_encoded.csv')

NameError: name 'pd' is not defined

In [37]:
qwerty.iloc[0]['encoding']

'tensor([-0.0022,  0.0454,  0.0139,  ...,  0.0002, -0.0323,  0.0227])'

In [2]:
import torch

In [7]:
len(torch.load('data/processed/wdbench/wdbench_nl_encoding.pt')[0])

4096